In [63]:
import json
import numpy as np
import datetime

In [ ]:


def read_input_file(input_file_name:str):
    '''Given a file, it returns the numpy
    '''
    with open(input_file_name, 'r') as file:
        data = json.load(file)
    return data

def calculate_covariance_matrix(input_data):
    '''Given a '''
    pass

In [7]:
data = read_input_file("../eth_hackathon/input_one_day.json")

In [ ]:
assets = list(data['assets'].keys())
# number of history elements (assumed to be equal for all assets)
num_history_dates = len(data['assets'][assets[0]]['history'])

# number of assets we are taking into account
N_considered_assets = 4 

# Each row of assets_matrix represents a variable, and each column a single observation of all those variables.
assets_matrix = np.empty((N_considered_assets, num_history_dates))

for i, asset in enumerate(assets[:N_considered_assets]):
    history = list(data['assets'][asset]['history'].values())
    ##TODO add filtering by date
    assets_matrix[i, :] = history

print(np.corrcoef(assets_matrix))

[[ 1.         -0.01979352 -0.14791098  0.32146658]
 [-0.01979352  1.          0.87165607 -0.7775364 ]
 [-0.14791098  0.87165607  1.         -0.81925307]
 [ 0.32146658 -0.7775364  -0.81925307  1.        ]]
